In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

Age_Cat_folders = [
    "C:/Users/OWNER/Downloads/Alabi/Computer Vision/Self Imple/Age Detection System/Dataset/6-20",
    "C:/Users/OWNER/Downloads/Alabi/Computer Vision/Self Imple/Age Detection System/Dataset/25-30",
    "C:/Users/OWNER/Downloads/Alabi/Computer Vision/Self Imple/Age Detection System/Dataset/42-48",
    "C:/Users/OWNER/Downloads/Alabi/Computer Vision/Self Imple/Age Detection System/Dataset/60-98"
    ]

def load_data(Age_Cat_folders):
    images = []
    labels = []
    for i, folder in enumerate(Age_Cat_folders):
        for filename in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (48,48))
                images.append(img)
                labels.append(i)
            except Exception as e:
                print(f"Error loading image {os.path.join(folder, filename)}:{e}")
    return np.array(images), np.array(labels)

images, labels = load_data(Age_Cat_folders)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1).astype('float32')/255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_test, y_test))
model.save("age_detection_model.h5")

C:\Users\OWNER\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 16s 123ms/step - accuracy: 0.4074 - loss: 1.3724 - val_accuracy: 0.5780 - val_loss: 0.9643
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 14s 124ms/step - accuracy: 0.6254 - loss: 0.9116 - val_accuracy: 0.6615 - val_loss: 0.8281
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 15s 131ms/step - accuracy: 0.6738 - loss: 0.8000 - val_accuracy: 0.6632 - val_loss: 0.7978
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 15s 129ms/step - accuracy: 0.7111 - loss: 0.7208 - val_accuracy: 0.7022 - val_loss: 0.7169
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 18s 156ms/step - accuracy: 0.7289 - loss: 0.6718 - val_accuracy: 0.7060 - val_loss: 0.7058
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 15s 129ms/step - accuracy: 0.7500 - loss: 0.6089 - val_accuracy: 0.7170 - val_loss: 0.6835
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 15s 130ms/step - accuracy: 0.7791 - loss: 0.5534 - val_accuracy: 0.7110 - val_loss: 0.6868
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 14s 125ms/step - accuracy: 0.7920 - loss: 0